In [7]:
import torch
from kan import *
from my_functions import calculate_signature, calculate_signature_over_all_surface, format_signature
import plotly.graph_objects as go


In [58]:
mesh_path = "data/bunny_curvs.ply"
dataset = create_dataset_from_mesh(mesh_path=mesh_path,sampled_index=20000, show_sample=True, ball_radius=0.06, down_sampling_ratio=0.8)
x = torch.tensor([[0.0], [0.0]], requires_grad=True).T
dataset['train_input'].shape, dataset['train_label'].shape

(torch.Size([97, 2]), torch.Size([97, 1]))

In [76]:
experiment_num = 10
signatures = torch.zeros(experiment_num, 6)
signatures2 = torch.zeros(experiment_num, 6)
dataset_sampled_points = []
grid_range = 0.08
ball_radius = 0.08
lr = 1.0
for i in range(experiment_num):
    dataset = create_dataset_from_mesh(mesh_path=mesh_path,sampled_index=20000, show_sample=False, ball_radius=ball_radius, seed=i, down_sampling_ratio=0.8)
    model = KAN(width=[2,5,1], grid=3, k=5, grid_eps=0.0, seed=0, grid_range=[-grid_range,grid_range], noise_scale=0.1, noise_scale_base=0.1, base_fun=torch.nn.SiLU())
    model.train(dataset, opt="LBFGS", steps=20,lr=lr, lamb=0.01, lamb_l1=1.0, lamb_entropy=1.0, lamb_coef=0.0, lamb_coefdiff=0.0, sglr_avoid=True)
    model = model.prune(threshold=3e-2)
    model.train(dataset, opt="LBFGS", steps=50,lr=lr, sglr_avoid=True)
    lib = ['x','x^2', 'x^3', 'x^4']
    model.auto_symbolic(lib=lib)
    model.train(dataset, opt="LBFGS", steps=50,lr=lr, sglr_avoid=True)
    print(model.symbolic_formula()[0][0])
    signature = calculate_signature(model, x) # only calc sig in the origin
    # signature2 = calculate_signature_over_all_surface(model, torch.tensor(dataset['train_input'], requires_grad=True)) # calc sig over all surface and take the mean
    print("exp ", i, "signature ", signature)
    signatures[i] = signature
    # signatures2[i] = signature2
    curr_sampled_points = torch.concatenate((dataset['train_input'], model(dataset['train_input'])), dim=1)
    dataset_sampled_points.append(curr_sampled_points)


train loss: 1.16e-02 | test loss: 1.31e-02 | reg: 3.58e+00 : 100%|██| 20/20 [00:01<00:00, 11.25it/s]
train loss: 4.39e-03 | test loss: 4.41e-02 | reg: 5.73e+00 : 100%|██| 50/50 [00:02<00:00, 19.99it/s]


fixing (0,0,0) with x^2, r2=0.8555208444595337
fixing (0,1,0) with x^3, r2=0.9819706082344055
fixing (1,0,0) with x^4, r2=0.1488385945558548


train loss: 6.47e-03 | test loss: 8.91e-03 | reg: 3.48e+02 : 100%|██| 50/50 [00:00<00:00, 52.99it/s]
/Users/gal.yona/Documents/MSc/Thesis/ImplicitNeuralSurfaces/forked_pykan/pykan/kan/KAN.py:325: UserWarning:

std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)



0.57 - 10135.83*(-(0.04 - x_2)**3 + 0.02*(-x_1 - 0.12)**2 - 0.09)**4
exp  0 signature  tensor([[  2.4279,  -6.4756,  -0.8251, -72.9593,   9.6303, 155.0416]],
       grad_fn=<StackBackward0>)


train loss: 1.16e-02 | test loss: 1.30e-02 | reg: 3.41e+00 : 100%|██| 20/20 [00:01<00:00, 11.76it/s]
train loss: 5.38e-03 | test loss: 2.05e-01 | reg: 2.08e+00 : 100%|██| 50/50 [00:02<00:00, 22.24it/s]


...

2363.66*(-0.02*(-x_1 - 0.1)**2 - (x_2 - 0.02)**3 - 0.17)**4 - 2.04
exp  8 signature  tensor([[ 2.4635e+00, -8.8555e+00, -3.3140e-01, -1.3601e+02,  5.6306e+00,
          3.7560e+02]], grad_fn=<StackBackward0>)


train loss: 1.18e-02 | test loss: 1.26e-02 | reg: 3.60e+00 : 100%|██| 20/20 [00:01<00:00, 11.60it/s]
train loss: 4.83e-03 | test loss: 2.95e-02 | reg: 1.42e+01 : 100%|██| 50/50 [00:02<00:00, 20.29it/s]


fixing (0,0,0) with x^2, r2=0.8465535640716553
fixing (0,1,0) with x^4, r2=0.9158629179000854
fixing (1,0,0) with x^4, r2=0.3389515280723572


train loss: 5.66e-03 | test loss: 8.90e-03 | reg: 2.67e+01 : 100%|██| 50/50 [00:00<00:00, 60.43it/s]
/Users/gal.yona/Documents/MSc/Thesis/ImplicitNeuralSurfaces/forked_pykan/pykan/kan/KAN.py:325: UserWarning:

std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)



1.40285051498182e+15*(0.01 - x_2)**16 - 0.03
exp  9 signature  tensor([[   2.4841,    5.0381,   13.1048, -260.0707,   96.9176, -733.6687]],
       grad_fn=<StackBackward0>)


In [77]:
# plot the sampled points in the same plot but different colors
fig = go.Figure()
for i in range(len(dataset_sampled_points)):
    curr_sampled_points = dataset_sampled_points[i].detach()
    fig.add_trace(go.Scatter3d(x=curr_sampled_points[:,0], y=curr_sampled_points[:,1], z=curr_sampled_points[:,2], mode='markers', marker=dict(size=3), name="exp "+str(i)))
fig.show()

In [78]:
signatures

tensor([[ 2.4279e+00, -6.4756e+00, -8.2514e-01, -7.2959e+01,  9.6303e+00,
          1.5504e+02],
        [-1.7151e+00,  1.9112e-01, -1.8927e+02, -1.8745e+02,  5.1012e+01,
          5.0390e+01],
        [ 2.2079e+00, -5.9002e+00, -1.7539e-01, -6.0215e+01,  2.1558e+00,
          1.2447e+02],
        [ 2.6790e+00, -4.7737e+00,  8.2494e-02, -7.1415e+01, -2.0100e+00,
          1.0603e+02],
        [ 2.2305e+00, -9.2930e+00, -7.2216e-01, -1.0488e+02,  1.6406e+01,
          3.2011e+02],
        [-3.4335e+00, -7.5803e-01, -2.1877e+02, -2.1484e+02, -2.0093e+02,
         -1.8428e+02],
        [ 2.5354e+00, -6.0884e+00, -1.9523e-01, -7.5877e+01,  1.9706e+00,
          1.4683e+02],
        [ 2.6282e+00, -4.4945e+00,  5.4628e-02, -7.0588e+01, -1.2205e+00,
          1.0129e+02],
        [ 2.4635e+00, -8.8555e+00, -3.3140e-01, -1.3601e+02,  5.6306e+00,
          3.7560e+02],
        [ 2.4841e+00,  5.0381e+00,  1.3105e+01, -2.6007e+02,  9.6918e+01,
         -7.3367e+02]], grad_fn=<CopySlices>)

In [79]:
 #discard all nan and 0
signatures = signatures[~torch.isnan(signatures).any(dim=1)]
signatures = signatures[~(signatures == 0).all(dim=1)]

In [80]:
mean_signature = torch.mean(signatures, dim=0)
std_signature = torch.std(signatures, dim=0)
print("mean signature ", mean_signature)
print("std signature ", std_signature)



mean signature  tensor([   1.4508,   -4.1410,  -39.7050, -125.4308,   -2.0437,   46.1814],
       grad_fn=<MeanBackward1>)
std signature  tensor([  2.1649,   4.4234,  86.9820,  71.4068,  76.5460, 312.6187],
       grad_fn=<StdBackward0>)


In [81]:
#discard all nan and 0
# signatures2 = signatures2[~torch.isnan(signatures2).any(dim=1)]
# signatures2 = signatures2[~(signatures == 0).all(dim=1)]
# mean_signature2 = torch.mean(signatures2, dim=0)
# std_signature2 = torch.std(signatures2, dim=0)
# print("mean signature ", mean_signature2)
# print("std signature ", std_signature2)


In [82]:
# remove outliers that are 2 stds away from the mean
outliers = torch.abs(signatures - mean_signature) > 2*std_signature
signatures_without_outliers = signatures[~outliers.any(dim=1)]
mean_signature = torch.mean(signatures_without_outliers, dim=0)
std_signature = torch.std(signatures_without_outliers, dim=0)
print("mean signature ", mean_signature)
print("std signature ", std_signature)
print(signatures_without_outliers.shape)
print(signatures_without_outliers)



mean signature  tensor([  1.9322,  -5.7112, -23.9226, -97.4244,  10.4468, 172.4703],
       grad_fn=<MeanBackward1>)
std signature  tensor([  1.4833,   2.9441,  66.8106,  43.9045,  17.4715, 113.8323],
       grad_fn=<StdBackward0>)
torch.Size([8, 6])
tensor([[ 2.4279e+00, -6.4756e+00, -8.2514e-01, -7.2959e+01,  9.6303e+00,
          1.5504e+02],
        [-1.7151e+00,  1.9112e-01, -1.8927e+02, -1.8745e+02,  5.1012e+01,
          5.0390e+01],
        [ 2.2079e+00, -5.9002e+00, -1.7539e-01, -6.0215e+01,  2.1558e+00,
          1.2447e+02],
        [ 2.6790e+00, -4.7737e+00,  8.2494e-02, -7.1415e+01, -2.0100e+00,
          1.0603e+02],
        [ 2.2305e+00, -9.2930e+00, -7.2216e-01, -1.0488e+02,  1.6406e+01,
          3.2011e+02],
        [ 2.5354e+00, -6.0884e+00, -1.9523e-01, -7.5877e+01,  1.9706e+00,
          1.4683e+02],
        [ 2.6282e+00, -4.4945e+00,  5.4628e-02, -7.0588e+01, -1.2205e+00,
          1.0129e+02],
        [ 2.4635e+00, -8.8555e+00, -3.3140e-01, -1.3601e+02,  5.6306e+

In [75]:
# calculate std/mean to see the variation of the signature
std_over_mean = abs(std_signature/mean_signature)
print("std/mean ", std_over_mean)

std/mean  tensor([1.5431, 2.1829, 1.0866, 0.7093, 0.9364, 0.7524],
       grad_fn=<AbsBackward0>)
